In [194]:
import praw
import pandas as pd
import itertools
import yfinance as yf
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly_express as px

from IPython.display import HTML
import numpy as np
from matplotlib.dates import date2num , DateFormatter


In [299]:
df2 = pd.read_csv('Data/temp_datalab_records_linkedin_company.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.



In [300]:
df2.head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,website,entity_id,cusip,isin
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00,2015-09-14 00:00:00+00,NaN,NaN,NaN,NaN,NaN
1,58329,2015-09-15,Goldman Sachs,552862,38141,https://www.linkedin.com/company/1382,Investment Banking,2015-09-15 00:00:00+00,2015-09-15 00:00:00+00,NaN,NaN,NaN,NaN,NaN
2,58363,2015-09-16,United Technologies,59157,14982,https://www.linkedin.com/company/2426,Aviation & Aerospace,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
3,58366,2015-09-16,Novo Nordisk,336175,26448,https://www.linkedin.com/company/2227,Pharmaceuticals,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
4,58371,2015-09-16,"Lowe's Companies, Inc.",134255,62574,https://www.linkedin.com/company/4128,Retail,2015-09-16 00:00:00+00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN


In [301]:
df2['date_added'] = pd.to_datetime(df2['date_added'])

In [306]:
len(df2['company_name'][df2['employees_on_platform']==0].unique())

0

In [303]:
# We drop companies that have 0 employees at any point in time

df2.drop(df2[df2['employees_on_platform']==0].index, inplace=True)

In [304]:
df2

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,website,entity_id,cusip,isin
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00:00,2015-09-14 00:00:00+00,NaN,NaN,NaN,NaN,NaN
1,58329,2015-09-15,Goldman Sachs,552862,38141,https://www.linkedin.com/company/1382,Investment Banking,2015-09-15 00:00:00+00:00,2015-09-15 00:00:00+00,NaN,NaN,NaN,NaN,NaN
2,58363,2015-09-16,United Technologies,59157,14982,https://www.linkedin.com/company/2426,Aviation & Aerospace,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
3,58366,2015-09-16,Novo Nordisk,336175,26448,https://www.linkedin.com/company/2227,Pharmaceuticals,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
4,58371,2015-09-16,"Lowe's Companies, Inc.",134255,62574,https://www.linkedin.com/company/4128,Retail,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426191,902985,2018-07-17,"Diversified Restaurant Holdings, Inc (Nasdaq: ...",488,89,https://www.linkedin.com/company/428083,Restaurants,2018-07-17 17:17:08.889700+00:00,2018-07-17 17:17:08.889725+00,NaN,http://www.diversifiedrestaurantholdings.com,NaN,NaN,NaN
2426192,903052,2018-07-17,First Financial Northwest Bank,785,118,https://www.linkedin.com/company/497673,Banking,2018-07-17 15:54:35.807059+00:00,2018-07-17 15:54:35.807085+00,NaN,http://www.ffnwb.com,NaN,NaN,NaN
2426193,902948,2018-07-17,DMC Global Inc.,1521,184,https://www.linkedin.com/company/586890,Oil & Energy,2018-07-17 13:44:19.302482+00:00,2018-07-17 13:44:19.302508+00,NaN,http://www.dmcglobal.com,NaN,NaN,NaN
2426194,910210,2018-07-17,ServiceSource,27963,2496,https://www.linkedin.com/company/11889,Information Technology and Services,2018-07-17 17:28:48.430667+00:00,2018-07-17 17:28:48.430717+00,NaN,http://www.servicesource.com,NaN,NaN,NaN


In [307]:
# We look at the number of employees at the earliest time and at the latest

df_latest = df2.loc[df2.groupby('dataset_id').date_added.idxmax()]
df_earliest = df2.loc[df2.groupby('dataset_id').date_added.idxmin()]

In [308]:
df_latest = df_latest.merge(df_earliest[['dataset_id', 'employees_on_platform']], suffixes=['_latest', '_earliest'], on='dataset_id')

In [309]:
df_latest['employees_ratio'] = df_latest['employees_on_platform_latest']/df_latest['employees_on_platform_earliest']

In [311]:
#Latest number of followers

fig = go.Figure()
fig.add_trace(go.Histogram(x=df_latest['employees_on_platform_latest'], nbinsx=100))

fig.show()

In [183]:
# Examine the bin between 0 and 10k employees

In [312]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_latest['employees_on_platform_latest'][df_latest['employees_on_platform_latest']<10000], nbinsx=250))

fig.show()

In [313]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_latest['employees_ratio'], nbinsx=100))

fig.show()

In [314]:
df_latest['industry'][pd.isnull(df_latest['industry'])] = ''

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [315]:
industries = df_latest['industry'].unique()
for industry in industries:
    aux = df_latest['employees_ratio'][df_latest['industry']==industry]
    if len(aux)>100:
        print(industry+": ", len(aux), " - ", np.mean(aux))

Financial Services:  260  -  1.4553410704428285
Computer Software:  166  -  1.2845172899907995
Pharmaceuticals:  175  -  1.3354863970280841
Electrical/Electronic Manufacturing:  107  -  1.7370270767652098
Biotechnology:  310  -  1.3904542562066455
Retail:  156  -  1.518242492903392
Information Technology and Services:  174  -  1.2260582985167168
Oil & Energy:  217  -  1.35780039751991
Telecommunications:  120  -  1.3635540561572
Medical Devices:  120  -  1.3021496007438347
Banking:  313  -  1.3175739777226663
Internet:  144  -  1.4801183825753725
Insurance:  104  -  1.3276159925352577
Semiconductors:  102  -  1.2437516707190175
Real Estate:  143  -  1.289229153637651


In [316]:
fig = go.Figure()
for industry in industries:
    aux = df_latest['employees_ratio'][df_latest['industry']==industry]
    if len(aux)>150:
        print(industry+": ", len(aux), " - ", np.mean(aux))
        fig.add_trace(go.Histogram(x=aux, histnorm='probability', name=industry))
fig.show()

Financial Services:  260  -  1.4553410704428285
Computer Software:  166  -  1.2845172899907995
Pharmaceuticals:  175  -  1.3354863970280841
Biotechnology:  310  -  1.3904542562066455
Retail:  156  -  1.518242492903392
Information Technology and Services:  174  -  1.2260582985167168
Oil & Energy:  217  -  1.35780039751991
Banking:  313  -  1.3175739777226663


In [ ]:
# When zooming in it's interesting to see that areas such as IT and computer services have grown at a slower rate than banking or retail.

In [317]:
fig = px.scatter(x=df_latest['employees_ratio'], 
                 y=df_latest['employees_on_platform_earliest'],
                hover_name=df_latest["company_name"], color=df_latest['industry'] , color_continuous_scale="Viridis")#)
fig.update_layout(xaxis = go.layout.XAxis(title_text = "Employees Ratio"),
                  yaxis = go.layout.YAxis(title_text = "Number of Employees (latest)"))
fig.show()

In [318]:
# We are going to look at a few stocks. 
# We first find the 10 stocks with the highest number of employees according to their latest data

top10 = df_latest.sort_values(by='employees_on_platform_latest').tail(10)
df_latest.sort_values(by='employees_on_platform_latest').head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform_latest,link,industry,date_added,date_updated,description,website,entity_id,cusip,isin,employees_on_platform_earliest,employees_ratio
3621,1578441,2018-07-17,Box Butler (acquired by Iron Mountain),133,1,https://www.linkedin.com/company/1604635,Consumer Services,2018-07-17 19:10:05.141679+00:00,2018-07-17 19:10:05.14173+00,NaN,http://www.boxbutler.com,NaN,NaN,NaN,2,0.5
4035,2094831,2018-02-17,New Residential Investment Corp.,291,1,https://www.linkedin.com/company/3102257,Real Estate,2018-02-17 19:25:12+00:00,2018-02-17 19:25:12+00,New Residential Investment Corp. (NYSE: NRZ) i...,NaN,NaN,NaN,NaN,1,1.0
4316,2200155,2018-02-17,2020b LLC,8,1,https://www.linkedin.com/company/1494452,Capital Markets,2018-02-17 19:25:12+00:00,2018-02-17 19:25:12+00,Today's exceptional challenges provide us with...,NaN,NaN,NaN,NaN,1,1.0
3852,2059443,2018-02-17,Access Globia Inc,0,1,https://www.linkedin.com/company/8486222,,2018-02-17 19:25:12+00:00,2018-02-17 19:25:12+00,NaN,NaN,NaN,NaN,NaN,1,1.0
3511,1342060,2018-07-17,"ApplianceSmart, Inc.",57,1,https://www.linkedin.com/company/3667618,Retail,2018-07-17 13:31:27.063946+00:00,2018-07-17 13:31:27.063967+00,NaN,http://www.ApplianceSmart.com,NaN,NaN,NaN,1,1.0


In [319]:
df_top10 = df2[df2['company_name'].isin(top10['company_name'])]

In [246]:
fig = px.area(x = df_top10['date_added'], 
              y = df_top10['employees_on_platform'], 
              facet_col=df_top10['company_name'], facet_col_wrap=2)
fig.show()

In [320]:
companies_top10 = top10['company_name'].unique()

In [321]:
companies_top10

array(['GE', 'Wells Fargo', 'Cognizant', 'Amazon', 'Bank of America',
       'JPMorgan Chase & Co.', 'Tata Consultancy Services', 'Accenture',
       'Walmart', 'IBM'], dtype=object)

In [322]:
fig = go.Figure()

for company in companies_top10:
    df_aux = df_top10.groupby('company_name').get_group(company)
    fig.add_traces(go.Scatter(x=df_aux["date_added"], y=df_aux["employees_on_platform"], name=company))
    #print(df_top10.groupby('company_name').get_group(company))
#fig.update_xaxes(
 #   dtick="M1",
  #  tickformat="%b\n%Y")
fig.show()

In [102]:
f = df2_test.sort_values('dataset_id').groupby('dataset_id')

In [105]:
f.head(20)

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,website,entity_id,cusip,isin
0,58329,2015-09-14,Goldman Sachs,552254,38124,https://www.linkedin.com/company/1382,Investment Banking,2015-09-14 00:00:00+00:00,2015-09-14 00:00:00+00,NaN,NaN,NaN,NaN,NaN
2459,58329,2015-09-22,Goldman Sachs,557322,35301,https://www.linkedin.com/company/1382,Investment Banking,2015-09-22 00:00:00+00:00,2015-09-22 00:00:00+00,NaN,NaN,NaN,NaN,NaN
280,58329,2015-09-17,Goldman Sachs,554209,38143,https://www.linkedin.com/company/1382,Investment Banking,2015-09-17 00:00:00+00:00,2015-09-17 00:00:00+00,NaN,NaN,NaN,NaN,NaN
9345,58329,2015-10-05,Goldman Sachs,565939,35380,https://www.linkedin.com/company/1382,Investment Banking,2015-10-05 00:00:00+00:00,2015-10-05 00:00:00+00,NaN,NaN,NaN,NaN,NaN
2973,58329,2015-09-23,Goldman Sachs,558082,35328,https://www.linkedin.com/company/1382,Investment Banking,2015-09-23 00:00:00+00:00,2015-09-23 00:00:00+00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7481,59374,2015-10-02,Freshpet,1257,71,https://www.linkedin.com/company/2389955,Consumer Goods,2015-10-02 00:00:00+00:00,2015-10-02 00:00:00+00,NaN,NaN,NaN,NaN,NaN
8568,59374,2015-10-04,Freshpet,1258,71,https://www.linkedin.com/company/2389955,Consumer Goods,2015-10-04 00:00:00+00:00,2015-10-04 00:00:00+00,NaN,NaN,NaN,NaN,NaN
6937,59374,2015-10-01,Freshpet,1255,71,https://www.linkedin.com/company/2389955,Consumer Goods,2015-10-01 00:00:00+00:00,2015-10-01 00:00:00+00,NaN,NaN,NaN,NaN,NaN
9528,59374,2015-10-06,Freshpet,1259,71,https://www.linkedin.com/company/2389955,Consumer Goods,2015-10-06 00:00:00+00:00,2015-10-06 00:00:00+00,NaN,NaN,NaN,NaN,NaN


In [323]:
df_stock = pd.read_csv('Data/companies.csv')

In [360]:
# Find ticker for company:
df_top10

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,link,industry,date_added,date_updated,description,website,entity_id,cusip,isin
8,58434,2015-09-16,IBM,2542182,414139,https://www.linkedin.com/company/1009,Information Technology and Services,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
13,58402,2015-09-16,GE,1017650,176829,https://www.linkedin.com/company/1015,Electrical/Electronic Manufacturing,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
25,58443,2015-09-16,Amazon,1311936,84671,https://www.linkedin.com/company/1586,Internet,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
37,58408,2015-09-16,Walmart,600109,229631,https://www.linkedin.com/company/2646,Retail,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
73,58472,2015-09-16,Bank of America,624321,198003,https://www.linkedin.com/company/1123,Banking,2015-09-16 00:00:00+00:00,2015-09-16 00:00:00+00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424232,58394,2018-07-17,Accenture,3409892,380166,https://www.linkedin.com/company/1033,Information Technology and Services,2018-07-17 17:58:07.262319+00:00,2018-07-17 17:58:07.262359+00,NaN,http://www.accenture.com,NaN,NaN,NaN
2424501,58402,2018-07-17,GE,2035998,235691,https://www.linkedin.com/company/1015,Electrical/Electronic Manufacturing,2018-07-17 18:54:17.453942+00:00,2018-07-17 18:54:17.453968+00,NaN,http://www.ge.com,NaN,NaN,NaN
2425010,58472,2018-07-17,Bank of America,947701,261629,https://www.linkedin.com/company/1123,Banking,2018-07-17 18:10:46.537336+00:00,2018-07-17 18:10:46.537386+00,NaN,https://www.bankofamerica.com,NaN,NaN,NaN
2425472,58398,2018-07-17,Wells Fargo,663990,240793,https://www.linkedin.com/company/1235,Financial Services,2018-07-17 20:27:21.629605+00:00,2018-07-17 20:27:21.629631+00,NaN,http://www.wellsfargo.com,NaN,NaN,NaN


In [326]:
gs = yf.Ticker("gs")

In [327]:
df_gs = gs.history(period="6y")

In [353]:
df_gs = df_gs[df_gs['Date']<'31-07-2019']

In [354]:
df_gs['Date'] = df_gs.index

In [355]:
lkd_gs = df2[df2['dataset_id']==58329]

In [357]:
def plot_sotck_employees(stock_df, lkd_df):

    trace1 = (go.Scatter(x=stock_df['Date'], y=stock_df["Open"], name='Open',
                        yaxis='y1'))
    trace2 = (go.Scatter(x=stock_df['Date'], y=stock_df["Close"], name="Close",
                        yaxis='y1'))
    trace3 = (go.Scatter(x=lkd_df['as_of_date'], y=lkd_df['employees_on_platform'],
                        yaxis='y2', name="Employees"))
    data = [trace1, trace2, trace3]
    layout = go.Layout(title= lkd_df['company_name'].iloc[0],
                       yaxis=dict(title='Share value'),
                       yaxis2=dict(title='Number of Employees',
                                   overlaying='y',
                                   side='right'))
    return go.Figure(data=data, layout=layout)
    

In [358]:
plot_sotck_employees(df_gs, lkd_gs)